In [1]:
import pywbemReq

c = pywbemReq.WBEMConnection('https://10.207.120.119:5989', ('Local/admin', 'Password123!'), 'root/emc/smis')

In [17]:
%timeit -n10 nl.EnumerateInstances('CIM_RegisteredProfile')

10 loops, best of 3: 2.74 s per loop


In [11]:
import datetime as dt

def timeit(func):
    def _inner(*args, **kwargs):
        start = dt.datetime.now()
        func(*args, **kwargs)
        end = dt.datetime.now()
        print('Time used: {} sec.'.format(end-start))
    
    return _inner

In [19]:
@timeit
def check_assoc(c, left, right, via):
    print(
        'Checking the associations from `{}` to `{}` via `{}`'.format(
            left, right, via))
    
    left_inss = c.EnumerateInstances(left)
    left_names = c.EnumerateInstanceNames(left)
    assert len(left_inss) == len(left_names)
    print('Enumerate of `{}`: OK.'.format(left))
    
    for left_ins in left_inss:
        right_inss = c.Associators(left_ins.path, AssocClass=via, ResultClass=right)
        for right_ins in right_inss:
            c.GetInstance(right_ins.path)
            re_left_inss = c.Associators(right_ins.path, AssocClass=via,
                                         ResultClass=left)
            assert left_ins in re_left_inss
            
    print(
        'Associations from `{}` to `{}` via `{}`: OK.'.format(
            left, right, via))

In [20]:
check_assoc(jc, 'CIM_RegisteredProfile', 'CIM_RegisteredProfile', 'CIM_ReferencedProfile')


Checking the associations from `CIM_RegisteredProfile` to `CIM_RegisteredProfile` via `CIM_ReferencedProfile`
Enumerate of `CIM_RegisteredProfile`: OK.


SSLError: ("bad handshake: Error([('rsa routines', 'INT_RSA_VERIFY', 'wrong signature length'), ('SSL routines', 'ssl3_get_key_exchange', 'bad signature')],)",)